# Lesson 5: Async Streaming, Executors & Multi-modal

Master async operations, streaming, tool execution strategies, and multi-modal content:

- ✅ Async tool definition with `async def` and `await`
- ✅ Streaming progress updates with `yield` in async tools
- ✅ Using `agent.stream_async()` for real-time responses
- ✅ ConcurrentToolExecutor for parallel tool execution (default)
- ✅ SequentialToolExecutor for ordered tool execution
- ✅ Multi-modal content (images, PDFs, documents)

**Estimated time:** 4-5 hours

**What you'll build:** Streaming tools, executor comparisons, and multi-modal examples!

## Setup

Import necessary modules and configure the environment:

In [1]:
import asyncio
import time
from datetime import datetime

from strands import Agent, tool
from strands.tools.executors import ConcurrentToolExecutor, SequentialToolExecutor

from lesson_utils import (
    load_environment,
    create_working_model,
    check_api_keys,
    print_troubleshooting,
)

# Load environment and check API keys
load_environment()
check_api_keys()

print("🎯 Lesson 5: Async Streaming, Executors & Multi-modal")
print("=" * 60)

✅ API keys detected: OpenAI
🎯 Lesson 5: Async Streaming, Executors & Multi-modal


## Part 1: Async Tools with Streaming Progress

Async tools can `yield` intermediate results to provide real-time progress updates. Each yielded value becomes a streaming event that you can consume with `agent.stream_async()`.

**Reference:** [Python Tools - Tool Streaming](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/#tool-streaming)

In [2]:
# Define async tools with yield for streaming progress

@tool
async def process_dataset(records: int) -> str:
    """Process records with progress updates."""
    start = datetime.now()

    for i in range(1, records + 1):
        await asyncio.sleep(0.1)  # Simulate processing time
        if i % 10 == 0:
            elapsed = (datetime.now() - start).total_seconds()
            yield f"Processed {i}/{records} records in {elapsed:.1f}s"

    total_time = (datetime.now() - start).total_seconds()
    yield f"✓ Completed {records} records in {total_time:.1f}s"


@tool
async def download_file(url: str, size_mb: int = 10) -> str:
    """Simulate downloading a file with progress updates."""
    chunks = 10
    chunk_size = size_mb / chunks

    yield f"Starting download: {url} ({size_mb}MB)"

    for i in range(1, chunks + 1):
        await asyncio.sleep(0.2)  # Simulate download time
        progress = (i / chunks) * 100
        downloaded = chunk_size * i
        yield f"Downloaded {downloaded:.1f}MB / {size_mb}MB ({progress:.0f}%)"

    yield f"✓ Download complete: {url}"


print("✅ Async streaming tools created!")

✅ Async streaming tools created!


In [3]:
model = create_working_model()

if model:
    # Create agent with async streaming tools
    agent = Agent(
        model=model,
        tools=[process_dataset, download_file],
        system_prompt="You are a helpful assistant with data processing capabilities.",
    )

    print("Streaming progress from async tools...\n")

    async def demo_streaming():
        async for event in agent.stream_async("Process 30 records"):
            # Check for tool stream events (progress updates)
            if tool_stream := event.get("tool_stream_event"):
                if update := tool_stream.get("data"):
                    print(f"📊 Progress: {update}")

            # Check for final text response
            if "data" in event and not event.get("tool_stream_event"):
                print(f"🤖 Agent: {event['data']}", end="")

    await demo_streaming()
else:
    print_troubleshooting()

🚀 Using OpenAI gpt-4o-mini
Streaming progress from async tools...


Tool #1: process_dataset
📊 Progress: Processed 10/30 records in 1.0s
📊 Progress: Processed 20/30 records in 2.0s
📊 Progress: Processed 30/30 records in 3.0s
📊 Progress: ✓ Completed 30 records in 3.0s
I🤖 Agent: I have🤖 Agent:  have successfully🤖 Agent:  successfully processed🤖 Agent:  processed 🤖 Agent:  30🤖 Agent: 30 records🤖 Agent:  records in🤖 Agent:  in 🤖 Agent:  3🤖 Agent: 3 seconds🤖 Agent:  seconds.🤖 Agent: . If🤖 Agent:  If you🤖 Agent:  you need🤖 Agent:  need any🤖 Agent:  any further🤖 Agent:  further assistance🤖 Agent:  assistance,🤖 Agent: , feel🤖 Agent:  feel free🤖 Agent:  free to🤖 Agent:  to ask🤖 Agent:  ask!🤖 Agent: !

## Part 2: ConcurrentToolExecutor (Parallel Execution)

`ConcurrentToolExecutor` is the default executor. It executes multiple tools in parallel when the LLM requests multiple tools in a single response.

**Reference:** [Tool Executors](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/executors/)

In [4]:
# Define tools that simulate API calls

@tool
async def get_weather(city: str) -> str:
    """Get weather forecast for a city."""
    await asyncio.sleep(1.0)  # Simulate API call
    return f"Weather in {city}: Sunny, 72°F"


@tool
async def get_time(city: str) -> str:
    """Get current time in a city."""
    await asyncio.sleep(1.0)  # Simulate API call
    return f"Time in {city}: 2:30 PM"


@tool
async def get_population(city: str) -> str:
    """Get population of a city."""
    await asyncio.sleep(1.0)  # Simulate database query
    return f"Population of {city}: ~1.5 million"


print("✅ API simulation tools created!")

✅ API simulation tools created!


In [5]:
model = create_working_model()

if model:
    # ConcurrentToolExecutor is the default
    agent = Agent(
        model=model,
        tool_executor=ConcurrentToolExecutor(),
        tools=[get_weather, get_time, get_population],
        system_prompt="Use tools to answer user questions about cities.",
    )

    print("Testing ConcurrentToolExecutor (tools run in parallel)...\n")

    start_time = time.time()
    response = await agent.invoke_async(
        "What's the weather, time, and population in Seattle?"
    )
    elapsed = time.time() - start_time

    print(f"\n🤖 Agent: {response}")
    print(f"\n⚡ Total time: {elapsed:.2f}s")
    print("💡 With concurrent execution, 3 tools (each taking 1s) complete in ~1s!")
else:
    print_troubleshooting()

🚀 Using OpenAI gpt-4o-mini
Testing ConcurrentToolExecutor (tools run in parallel)...


Tool #1: get_weather

Tool #2: get_time

Tool #3: get_population
In Seattle, the weather is sunny with a temperature of 72°F. The current time is 2:30 PM, and the population of Seattle is approximately 1.5 million.
🤖 Agent: In Seattle, the weather is sunny with a temperature of 72°F. The current time is 2:30 PM, and the population of Seattle is approximately 1.5 million.


⚡ Total time: 4.68s
💡 With concurrent execution, 3 tools (each taking 1s) complete in ~1s!


## Part 3: SequentialToolExecutor (Ordered Execution)

`SequentialToolExecutor` executes tools one after another in the order specified by the LLM. This is useful for dependent operations where one tool's output is needed by the next.

In [6]:
# Define tools for dependent operations

@tool
async def take_screenshot(filename: str) -> str:
    """Take a screenshot and save to file."""
    await asyncio.sleep(0.5)  # Simulate screenshot capture
    return f"Screenshot saved to {filename}"


@tool
async def compress_file(filename: str) -> str:
    """Compress a file to save space."""
    await asyncio.sleep(0.5)  # Simulate compression
    compressed = filename.replace(".png", ".zip")
    return f"Compressed {filename} to {compressed}"


@tool
async def send_email(recipient: str, attachment: str) -> str:
    """Send an email with an attachment."""
    await asyncio.sleep(0.5)  # Simulate email sending
    return f"Email sent to {recipient} with attachment: {attachment}"


print("✅ Workflow tools created!")

✅ Workflow tools created!


In [7]:
model = create_working_model()

if model:
    # Use SequentialToolExecutor for dependent operations
    agent = Agent(
        model=model,
        tool_executor=SequentialToolExecutor(),
        tools=[take_screenshot, compress_file, send_email],
        system_prompt="Execute tasks in the correct order for dependent operations.",
    )

    print("Testing SequentialToolExecutor (tools run in order)...")
    print("Task: Take screenshot → Compress → Email\n")

    start_time = time.time()
    response = await agent.invoke_async(
        "Take a screenshot named report.png, compress it, "
        "then email the compressed file to boss@company.com"
    )
    elapsed = time.time() - start_time

    print(f"\n🤖 Agent: {response}")
    print(f"\n⏱️  Total time: {elapsed:.2f}s")
    print("💡 Operations executed in the correct order!")
else:
    print_troubleshooting()

🚀 Using OpenAI gpt-4o-mini
Testing SequentialToolExecutor (tools run in order)...
Task: Take screenshot → Compress → Email


Tool #1: take_screenshot

Tool #2: compress_file

Tool #3: send_email
The screenshot has been taken, compressed, and emailed to boss@company.com successfully.
🤖 Agent: The screenshot has been taken, compressed, and emailed to boss@company.com successfully.


⏱️  Total time: 5.04s
💡 Operations executed in the correct order!


## Part 4: Performance Comparison

Let's compare the performance difference between concurrent and sequential execution.

In [8]:
@tool
async def fetch_data(source: str) -> str:
    """Fetch data from a source (simulated)."""
    await asyncio.sleep(1.0)  # Simulate network delay
    return f"Data from {source}: [sample data]"


model = create_working_model()

if model:
    # Test with ConcurrentToolExecutor
    print("1. Testing ConcurrentToolExecutor:")
    concurrent_agent = Agent(
        model=model,
        tool_executor=ConcurrentToolExecutor(),
        tools=[fetch_data],
        system_prompt="Fetch data from multiple sources.",
    )

    start = time.time()
    await concurrent_agent.invoke_async("Fetch data from API-A, API-B, and API-C")
    concurrent_time = time.time() - start
    print(f"   ⚡ Concurrent time: {concurrent_time:.2f}s")

    # Test with SequentialToolExecutor
    print("\n2. Testing SequentialToolExecutor:")
    sequential_agent = Agent(
        model=model,
        tool_executor=SequentialToolExecutor(),
        tools=[fetch_data],
        system_prompt="Fetch data from multiple sources.",
    )

    start = time.time()
    await sequential_agent.invoke_async("Fetch data from API-A, API-B, and API-C")
    sequential_time = time.time() - start
    print(f"   ⏱️  Sequential time: {sequential_time:.2f}s")

    # Show comparison
    print("\n3. Performance Summary:")
    print(f"   Concurrent: {concurrent_time:.2f}s")
    print(f"   Sequential: {sequential_time:.2f}s")
    speedup = sequential_time / concurrent_time if concurrent_time > 0 else 1
    print(f"   Speedup: {speedup:.2f}x faster with concurrent execution!")
else:
    print_troubleshooting()

🚀 Using OpenAI gpt-4o-mini
1. Testing ConcurrentToolExecutor:

Tool #1: fetch_data

Tool #2: fetch_data

Tool #3: fetch_data
I have fetched the data from the sources:

- **API-A**: Data from API-A: [sample data]
- **API-B**: Data from API-B: [sample data]
- **API-C**: Data from API-C: [sample data]   ⚡ Concurrent time: 4.69s

2. Testing SequentialToolExecutor:

Tool #1: fetch_data

Tool #2: fetch_data

Tool #3: fetch_data
Here is the data fetched from the APIs:

- **API-A**: Data from API-A: [sample data]
- **API-B**: Data from API-B: [sample data]
- **API-C**: Data from API-C: [sample data]   ⏱️  Sequential time: 7.05s

3. Performance Summary:
   Concurrent: 4.69s
   Sequential: 7.05s
   Speedup: 1.50x faster with concurrent execution!


## Part 5: Multi-modal Content

Agents can process multi-modal content including images and PDFs. This enables visual understanding and document analysis.

**Reference:** [Multi-modal Example](https://strandsagents.com/latest/documentation/docs/examples/python/multimodal/)

In [9]:
# Multi-modal works with any vision model (OpenAI, Anthropic, etc.)
# We pass images directly in messages using Bedrock Converse format

import os

# Create sample receipt
def create_sample_receipt():
    from PIL import Image, ImageDraw, ImageFont
    
    width, height = 400, 600
    img = Image.new('RGB', (width, height), color='white')
    draw = ImageDraw.Draw(img)
    
    try:
        font_large = ImageFont.truetype("Arial.ttf", 24)
        font_medium = ImageFont.truetype("Arial.ttf", 18)
        font_small = ImageFont.truetype("Arial.ttf", 14)
    except IOError:
        font_large = ImageFont.load_default()
        font_medium = ImageFont.load_default()
        font_small = ImageFont.load_default()
    
    # Draw receipt (simplified)
    y = 20
    draw.text((width//2-70, y), "ACME GROCERY", fill='black', font=font_large)
    y += 40
    draw.text((width//2-60, y), "123 Main St", fill='black', font=font_small)
    y += 40
    draw.line([(20, y), (width-20, y)], fill='black', width=2)
    y += 20
    draw.text((20, y), "Receipt #: 2025-001234", fill='black', font=font_small)
    y += 25
    draw.text((20, y), "Date: Oct 11, 2025", fill='black', font=font_small)
    y += 40
    
    items = [
        ("Apples (2 lbs)", "$5.99"),
        ("Bread", "$3.49"),
        ("Eggs (12)", "$4.99"),
        ("Milk", "$4.29"),
        ("Salmon", "$12.99"),
        ("Greens", "$3.49"),
    ]
    for item, price in items:
        draw.text((20, y), item, fill='black', font=font_small)
        draw.text((width-80, y), price, fill='black', font=font_small)
        y += 22
    
    y += 10
    draw.line([(20, y), (width-20, y)], fill='black', width=1)
    y += 20
    draw.text((20, y), "Subtotal:", fill='black', font=font_medium)
    draw.text((width-85, y), "$35.24", fill='black', font=font_medium)
    y += 25
    draw.text((20, y), "Tax:", fill='black', font=font_medium)
    draw.text((width-85, y), "$3.00", fill='black', font=font_medium)
    y += 25
    draw.line([(20, y), (width-20, y)], fill='black', width=2)
    y += 20
    draw.text((20, y), "TOTAL:", fill='black', font=font_large)
    draw.text((width-90, y), "$38.24", fill='black', font=font_large)
    
    filename = 'sample_receipt.png'
    img.save(filename)
    return filename

print("📸 Creating sample receipt...")
receipt_path = create_sample_receipt()
print(f"✓ Created: {receipt_path}\n")

# Read image as bytes
with open(receipt_path, 'rb') as f:
    image_bytes = f.read()

model = create_working_model()

if model:
    print("Using provider-agnostic multi-modal approach\n")
    
    # Demo 1: Document Analyzer (works with any vision model)
    print("1. Document Analyzer:")
    analyzer = Agent(
        model=model,
        system_prompt="Analyze images and extract all visible text and key information."
    )
    
    # Pass image directly in message (Bedrock Converse format)
    message = [
        {"text": "Analyze this receipt image and extract key information:"},
        {"image": {"format": "png", "source": {"bytes": image_bytes}}}
    ]
    
    response = await analyzer.invoke_async(message)
    print(f"📄 Analysis:\n{response}\n")
    
    # Demo 2: Receipt Extractor
    print("2. Receipt Extractor:")
    extractor = Agent(
        model=model,
        system_prompt="Extract financial data: store, date, line items, totals."
    )
    
    message = [
        {"text": "Extract all financial data from this receipt:"},
        {"image": {"format": "png", "source": {"bytes": image_bytes}}}
    ]
    
    response = await extractor.invoke_async(message)
    print(f"💰 Data:\n{response}\n")
    
    # Cleanup
    os.remove(receipt_path)
    
    print("💡 Key Takeaways:")
    print("✓ Images passed directly in messages (provider-agnostic)")
    print("✓ Works with OpenAI, Anthropic, and other vision models")
    print("✓ Bedrock Converse format: {'image': {'format': 'png', ...}}")
else:
    print_troubleshooting()

📸 Creating sample receipt...
✓ Created: sample_receipt.png

🚀 Using OpenAI gpt-4o-mini
Using provider-agnostic multi-modal approach

1. Document Analyzer:
Here is the key information extracted from the receipt:

- **Store Name:** ACME GROCERY
- **Address:** 123 Main St
- **Receipt Number:** 2025-001234
- **Date:** October 11, 2025

**Items Purchased:**
1. Apples (2 lbs) - $5.99
2. Bread - $3.49
3. Eggs (12) - $4.99
4. Milk - $4.29
5. Salmon - $12.99
6. Greens - $3.49

**Financial Summary:**
- **Subtotal:** $35.24
- **Tax:** $3.00
- **Total:** $38.24📄 Analysis:
Here is the key information extracted from the receipt:

- **Store Name:** ACME GROCERY
- **Address:** 123 Main St
- **Receipt Number:** 2025-001234
- **Date:** October 11, 2025

**Items Purchased:**
1. Apples (2 lbs) - $5.99
2. Bread - $3.49
3. Eggs (12) - $4.99
4. Milk - $4.29
5. Salmon - $12.99
6. Greens - $3.49

**Financial Summary:**
- **Subtotal:** $35.24
- **Tax:** $3.00
- **Total:** $38.24


2. Receipt Extractor:
Here is 

## Experiments

Now it's your turn! Try these experiments:

### Exercises:
1. **Large dataset processing** - Process 100 records and watch the streaming progress
2. **Variable delays** - Create tools with different delays (0.5s, 1s, 2s) and compare executors
3. **Data pipeline** - Build a multi-step pipeline with dependent operations
4. **Parallel API calls** - Simulate calling 5 different APIs concurrently
5. **Real images** - Use different receipt/invoice images with the multi-modal approach
6. **Multiple images** - Pass multiple images in one message for comparison
7. **Document classification** - Build a classifier that categorizes document types

### Challenge:
Build a document processing pipeline that:
1. Downloads multiple files concurrently
2. Processes them sequentially with progress updates
3. Generates a summary report

Use the cell below for your experiments:

In [ ]:
# Your experiments here!


## ✅ Success Criteria

You've completed Lesson 5 if:

- ✅ Async tools stream progress in real-time via `yield`
- ✅ ConcurrentToolExecutor runs tools in parallel
- ✅ SequentialToolExecutor runs tools in order
- ✅ Async tools show measurable speedup vs sequential
- ✅ Stream events are properly formatted and received
- ✅ Agent processes images (PNG, JPEG) correctly
- ✅ Multi-modal agents invoked with real receipt image
- ✅ Document analyzer and receipt extractor demonstrated

## 💡 Key Concepts Learned

- **Async Tools** - Use `async def` with `yield` for streaming progress
- **Streaming** - `agent.stream_async()` for real-time event consumption
- **ConcurrentToolExecutor** - Parallel execution for independent operations (1.4-1.5x speedup)
- **SequentialToolExecutor** - Ordered execution for dependent operations
- **Multi-modal** - Process images with `image_reader` tool, extract text via OCR

## Next Steps

- **Lesson 6**: Hooks & Structured Output - Lifecycle hooks and Pydantic models
- **Lesson 7**: Advanced Tools, Context & MCP - Class-based tools, conversation management, MCP integration

Ready to continue? Open `lesson_06_hooks_structured.ipynb`!